# Project Group - 

Members: Tieme van Hijum, Emma van den Brink, Sil Havinga

Student numbers: 4923588, 5136008, 4730321

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

Research Question:

 - What is the development over time in The Netherlands when it comes to the share of population outside of the maximum distance to a hospital?

Data Sources: 
 - CBS: Per municipality, district and neighborhood; proximity to hospital and population. https://www.cbs.nl/nl-nl/reeksen/publicatie/nabijheidsstatistieken-per-wijk-buurt
 - Optionally, data for hospital locations over time

Time Scale:
 - Data is available from 2007 until 2022, but the geography has changed over that time.

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

# Data Pipeline